Dependences to install

In [13]:
%pip install tensorflow==2.3.1
%pip install gym
%pip install keras-rl2
%pip install gym[atari]

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.1 (from versions: 2.5.0rc0, 2.5.0rc1, 2.5.0rc2, 2.5.0rc3, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.9.0rc0)
ERROR: No matching distribution found for tensorflow==2.3.1


  Using cached gym-0.23.1-py3-none-any.whl
  Using cached gym_notices-0.0.6-py3-none-any.whl (2.7 kB)
Note: you may need to restart the kernel to use updated packages.

  Using cached ale_py-0.7.4-cp39-cp39-win_amd64.whl (904 kB)



Environment : OpenAI Gym

In [1]:
import random
import gym
gym.__file__


'C:\\Users\\AbdelfataH\\anaconda3\\envs\\gym\\lib\\site-packages\\gym\\__init__.py'

In [2]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [3]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [4]:
episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    scoure = 0
    while not done:
        env.render()
        action = random.choice([0, 1, 2, 3, 4, 5])
        n_state, reward, done, info = env.step(action)
        scoure += reward
    print('Episode: {} Scoure: {}'.format(episode, scoure))
env.close()

C:\Users\AbdelfataH\anaconda3\envs\gym\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Impossible de modifier le mode thread une fois qu’il a été fixé
  warnings.warn(str(err))


Episode: 1 Scoure: 105.0
Episode: 2 Scoure: 255.0
Episode: 3 Scoure: 45.0
Episode: 4 Scoure: 105.0
Episode: 5 Scoure: 180.0


Deep Learning Model : Keras

In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

ModuleNotFoundError: No module named 'tensorflow'

Build Agent : Keras-RL

Reloading Agent from Memory